# Random Forest Classifier

Using cleaned data, we combined the dataset with converted dates of the accident, so that we could add time as an additional feature to predict severity of casualty.

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("./data.csv") 
time_data = pd.read_csv('./Road Accident/Road-Accident.csv', na_values=-1, parse_dates=['time'], infer_datetime_format=True)


/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3018: DtypeWarning: Columns (0,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
# config datetime to ordered integers
st = pd.Timestamp(year=1900, month=1, day=1)
def convert_date(x):
    return (pd.to_datetime(x,dayfirst=True) - st).days + 2
#print((pd.to_datetime('31/12/2014',dayfirst=True) - st).days + 2)
for i, row in time_data.iterrows():
    if len(row['date']) == 10:
        c = convert_date(row['date'])
        time_data.set_value(i,'date', c)
     
    
time_data['date'] = time_data['date'].astype(np.int64)

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:9: FutureWarning: set_value is deprecated and will be removed in a future release. Please use .at[] or .iat[] accessors instead
  if __name__ == '__main__':


In [4]:
# merge two datasets with common index, adding date column to data

d = data['Unnamed: 0'].values
dt = time_data.date.values
new_col = []
for i in d:
    new_col.append(dt[i])

data['date'] = new_col

data = data.dropna()


Preparing merged data into X_data and y_labels. We split the data into training and testing data (with a 80:20 ratio). We then used RandomForestClassifier to fit the model, and we analysed the results using a confusion matrix.

In [5]:
y = data.casualty_severity
X = data.drop('casualty_class', axis=1)

In [6]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [7]:
continuous_variables = ['longitude', 'latitude']

y = data['casualty_severity']
X = data.drop('casualty_severity', axis=1)

In [8]:
# Split data into training and testing data

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=4, test_size=0.2)

In [9]:
# Fit model into Classifier

rf = RandomForestClassifier(max_depth=4, n_estimators=50)
rf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=4, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [10]:
y_test_pred = rf.predict(X_test)

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

print("RESULTS: \n")

print("The confusion matrix: \n")
print(confusion_matrix(y_test, y_test_pred))
print("\nThe classification report:\n")
print(classification_report(y_test, y_test_pred, digits=3))

RESULTS: 

The confusion matrix: 

[[    0    30    42]
 [    0   397   835]
 [    0    16 11882]]

The classification report:

              precision    recall  f1-score   support

         1.0      0.000     0.000     0.000        72
         2.0      0.896     0.322     0.474      1232
         3.0      0.931     0.999     0.964     11898

   micro avg      0.930     0.930     0.930     13202
   macro avg      0.609     0.440     0.479     13202
weighted avg      0.923     0.930     0.913     13202



/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
